In [ ]:
from boardlaw.grid import *

In [ ]:
desc = 'main/5'
acks = acknowledged(desc)
for width in [1, 2, 4, 8, 16, 32]:
    for depth in [1, 2, 4, 8, 16, 32]:
        params = dict(width=width, depth=depth, boardsize=5, timelimit=45*60, desc=desc)
        if is_missing(params, acks):
            log.info(f'Launching {params}')
            jittens.jobs.submit(
                cmd='python -c "from boardlaw.main import *; run_jittens()" >logs.txt 2>&1',
                dir='.',
                resources={'gpu': 1},
                params=params)


In [ ]:
vast.jittenate(local=True)

In [ ]:
from IPython import display

vast.jittenate(local=True)
while not jittens.finished():
    display.clear_output(wait=True)
    jittens.refresh()
    time.sleep(1)

In [ ]:
jittens.refresh()

In [ ]:
jittens.manage.tails('output/pavlov/*/logs.0.txt')

In [ ]:
jittens.manage.cleanup()